In [28]:
import pandas as pd
import numpy as np
import csv

In [29]:
df = pd.read_csv('salario.csv')
salario_2 = df.drop('salario', axis=1)

In [30]:
dt2 = []
dt2 = salario_2["data"].str.split("/", n = 2, expand = True)
salario_2 = salario_2.drop("data", axis=1)

salario_2['dia'] = dt2[0]
salario_2['mes'] = dt2[1]
salario_2['ano'] = dt2[2]

In [31]:
cidades = salario_2.cidade.unique().tolist()
estados = salario_2.estado.unique().tolist()
paises = salario_2.pais.unique().tolist()
generos = salario_2.genero.unique().tolist()
dias = salario_2.dia.unique().tolist()
meses = salario_2.mes.unique().tolist()
anos = salario_2.ano.unique().tolist()

In [32]:
def teste(df, atributo, i): #df: dataframe, atributo: lista, i: indice do atributo na coluna
    colunas = ['id', 'pais', 'estado', 'cidade','genero', 'dia', 'mes', 'ano']
    count = 0
    grupos = []
    for at in atributo:
        atrib = df.groupby(colunas[i])
        atrib = atrib.get_group(at)
        list(atrib)
        count = count+1
        grupos.append([atrib])
         #print("Grupo {}: \n{}\n{}".format(count,atrib,"-"*100))
    return grupos

In [33]:
colunas = ['id', 'pais', 'estado', 'cidade','genero', 'dia', 'mes', 'ano']

In [34]:
teste(salario_2, dias,5)

[[      id            pais                      estado  \
  0      1   United States                     Alabama   
  5      6   United States                        Ohio   
  66    67        Portugal                       Porto   
  87    88        Portugal                      Aveiro   
  105  106          Mexico         Michoacan De Ocampo   
  122  123        Portugal            Viana do Castelo   
  187  188        Portugal            Viana do Castelo   
  196  197          Sweden                 Västmanland   
  202  203          Sweden             Västra Götaland   
  265  266          France               Île-de-France   
  272  273  United Kingdom                    Scotland   
  282  283        Portugal                       Braga   
  299  300        Portugal                    Bragança   
  350  351   United States              North Carolina   
  361  362        Portugal                     Setúbal   
  382  383          Canada                     Alberta   
  386  387    